# Protein and Genetic Engineering

### P2 - Scoring and sampling

#### Introduction

One essential aspect when optimizing protein structures is to define a search principle. In our case, this search principle will be the finding of the lowest energy structure. Therefore, it is vital to have a good scoring function if we want our method to succeed. A good scoring function is the one that behaves correctly in different modeling scenarios, from protein structure to experimental thermodynamic data prediction. 

In this practical session, we check how to evaluate a protein Pose with the Rosetta score function, and then we use this scoring method to apply a simple Monte-Carlo search of a tripeptide molecule. 

#### Importing and initializing Rosetta

First, we start by importing the library's content in our Jupyter notebook:

In [1]:
from pyrosetta import *
init()

PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python36.ubuntu 2020.19+release.f98ad046ef76418f1431e66d54e6074e2a0ec48c 2020-05-06T13:59:29] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.ubuntu r254 2020.19+release.f98ad046ef7 f98ad046ef76418f1431e66d54e6074e2a0ec48c http://www.pyrosetta.org 2020-05-06T13:59:29
core.init: command: PyRosetta -ex1 -ex2aro -database /home/martin/miniconda3/lib/python3.6/site-packages/pyrosetta-2020.19+release.f98ad046ef7-py3.6-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1772992587 seed_offset=0 real_seed=-1772992587
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-1772992587 RG_type=mt19937


### Access, create and modify a score function

We start by accessing the default Rosetta Score function (ref2015):

In [ ]:
sfxn = get_score_function(True)

Let's print the score function object to see details about it:

In [ ]:
print(sfxn)

We can get the weights of individual score terms by iterating all the non-zero score terms:

In [ ]:
for score_type in sfxn.get_nonzero_weighted_scoretypes():
    print(score_type, sfxn.get_weight(score_type))

For the definition of any of these score functions, you can check:

[Rosetta Score Types](http://new.rosettacommons.org/docs/latest/rosetta_basics/scoring/score-types)

We can modify any of these weights by calling the set_weigth method. Let's do this by first creating an empty score function object and then assigning weights for the 'fa_atr' and the 'fa_rep' terms.

In [ ]:
from pyrosetta.rosetta.core.scoring import *

In [ ]:
sfxn2 = ScoreFunction() # Empty score function
sfxn2.set_weight(fa_atr, 1.0)
sfxn2.set_weight(fa_rep, 1.0)

In [ ]:
for score_type in sfxn2.get_nonzero_weighted_scoretypes():
    print(score_type, sfxn2.get_weight(score_type))

### Scoring a Pose

Now, let's load a protein structure into a Pose object and get its score:

In [ ]:
pose = pose_from_pdb('input/6Q21.pdb')

First, we score the Pose with the default score function:

In [ ]:
print(sfxn(pose))

Then, we score the Pose with the two terms score function we just created:

In [ ]:
print(sfxn2(pose))

You can see that the score function values are different depending on the score function used to evaluate the Pose. We can see more details for the score function if we use the show() method:

In [ ]:
sfxn.show(pose)

In [ ]:
sfxn2.show(pose)

The scores for this Pose are high; it is usually recommended to minimize the structures before using them for any modeling protocol.

### Accessing per-residue scores

Rosetta's score function is decomposable by residues; we can access the specific energy a residue in the Pose has by calling the energies() method. Let-s first score the Pose with the default energy function:

In [ ]:
sfxn(pose)

Now let's see the energy for residue 24:

In [ ]:
pose.energies().show(24)

### Perturb Pose and evaluate its energy

As in the previous practice session, we will create an alanine tripeptide, and we are going to perturb its phi and psi angles randomly.

We start by creating the pose object:

In [ ]:
# Create a three peptide 
tripeptide = pose_from_sequence("AAA")

Let's create the Pymol Visualizer to load our tripeptide pose:

In [ ]:
pymol_mover = PyMOLMover()
pymol_mover.keep_history(True)

Now we send the tripeptide Pose to PyMol

In [ ]:
pymol_mover.apply(tripeptide)

Let's now score this tripeptide pose with our default all-atom energy function:

In [ ]:
sfxn.show(tripeptide)

We can store the total score of the pose into a variable to be able to access the energy value:

In [ ]:
energy = sfxn(tripeptide)
print(energy)

We now create a random phi or psi perturbation function for the pose. We make the function return the energy value of our modified pose:

In [ ]:
import numpy as np

In [ ]:
def perturb_random_angle(pose, max_rot=6):
    
    # Define the perturbation magnitude
    magnitude = np.random.uniform(low=-max_rot, high=max_rot)
    
    #Chose a random angle to perturb between phi and psi
    angle = np.random.choice(['phi', 'psi'])
    
    # Choose a random residue to perturb
    residues = range( 1 , pose.total_residue()  + 1 )
    residue = np.random.choice(residues)
    
    # Perturb the selected angle by the defined magnitude
    if angle == 'phi':
        orig_phi = pose.phi(residue)
        new_phi = orig_phi+magnitude
        
        # Keep phi value in the -180 tp 180 range
        if new_phi > 180:
            new_phi -= 360
        elif new_phi <= -180:
            new_phi += 360
        pose.set_phi(residue, new_phi)
        
    elif angle == 'psi':
        orig_psi = pose.psi(residue)
        new_psi = orig_psi+magnitude
        
        # Keep psi value in the -180 tp 180 range
        if new_psi > 180:
            new_psi -= 360
        elif new_psi <= -180:
            new_psi += 360
        pose.set_psi(residue, new_psi)

We can now perturb the pose a 1000 steps and check how the energy is progressing:

In [ ]:
# Redefine the tripeptide Pose
tripeptide = pose_from_sequence("AAA")

# Store energies into a list
energies = []

# Create pose to store best sampled result
best = Pose()
best.assign(tripeptide)
Eb = sfxn(best)

for i in range(100000):
    
    # Perturb and store energy
    perturb_random_angle(tripeptide)
    E = sfxn(tripeptide)
    energies.append(E)
    
    # Save pose if best stored result is lower in energy
    if E < Eb:
        best.assign(tripeptide)
        Eb = E

Let's plot the energy progression:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(energies)
plt.title('Random Sampling')
plt.xlabel('Step')
plt.ylabel('Energy [kcal/mol]')

Let's send the best sampled pose to PyMol:

In [ ]:
print('The best sampled score was: %.2f kcal/mol' % Eb)
pymol_mover.apply(best)

We repeat the process, but now we also save the phi and psi value to see which region of Ramachandran space our random perturbation of the pose is sampling. We create a new tripeptide and set the phi and psi angles to 90 for better visualization:

In [ ]:
# Redefine the tripeptide Pose
tripeptide = pose_from_sequence("AAA")

# Set phi and psi values to 90
tripeptide.set_phi(2, 90)
tripeptide.set_psi(2, 90)

# Store energies into a list
energies = []

# Store phi and psi torsion angles into a list
phi_values = []
psi_values = []

# Create pose to store best sampled result
best = Pose()
best.assign(tripeptide)
Eb = sfxn(best)

for i in range(100000):
    perturb_random_angle(tripeptide)
    E = sfxn(tripeptide)
    energies.append(E)
    
    # Store phi and psi angles
    phi_values.append(tripeptide.phi(2))
    psi_values.append(tripeptide.psi(2))
    
    # Save pose if best stored result is lower in energy
    if E < Eb:
        best.assign(tripeptide)
        Eb = E

Let's plot the energy:

In [ ]:
plt.plot(energies)
plt.title('Random Sampling')
plt.xlabel('Step')
plt.ylabel('Energy [kcal/mol]')

And the phi and psi values explored:

In [ ]:
# Define resolution and figure size
plt.figure(dpi=70, figsize=(4,4))

# Plot the phi and psi values as a scatter plot
plt.scatter(phi_values, psi_values, c='k', s=0.1)

# Generate labels for each axis
plt.xlabel('$\phi$')
plt.ylabel('$\psi$')

# Define the plot x and y limits
plt.xlim(-180,180)
plt.ylim(-180,180)

# Set a title
plt.title('Random exploration')

Finally, we send the best-sampled Pose to Pymol:

In [ ]:
print('The best sampled score was: %.2f kcal/mol' % Eb)
pymol_mover.apply(best)

### Minimization

A simple minimization protocol should get us to the nearest local minima in the energy landscape. We set a function that only accepts the mover if the energy is lower after it has been applied:

In [ ]:
def minimizer(pose, mover, score_function):
    
    # Get the current energy of the pose
    E0 = score_function(pose)
    
    # Create a copy of the pose
    clone_pose = Pose()
    clone_pose.assign(pose)
    
    # Apply perturbation to cloned pose
    mover(clone_pose)
    
    # Evaluate energy of the perturbed pose
    E1 = score_function(clone_pose)
    
    # Calculate the acceptance probability
    if E1-E0 < 0:
        pose.assign(clone_pose)

Let's run this function for 100000 steps to see how fast it converges into a local minimum.

In [ ]:
# Redefine the tripeptide Pose
tripeptide = pose_from_sequence("AAA")

# Set phi and psi values to 90
tripeptide.set_phi(2, 90)
tripeptide.set_psi(2, 90)

# Store energies into a list
energies = []

# Store phi and psi torsion angles into a list
phi_values = []
psi_values = []

# Create pose to store best sampled result
best = Pose()
best.assign(tripeptide)
Eb = sfxn(best)

for i in range(100000): 
    minimizer(tripeptide, perturb_random_angle, sfxn)
    
    E = sfxn(tripeptide)
    energies.append(E)
    
    phi_values.append(tripeptide.phi(2))
    psi_values.append(tripeptide.psi(2))
    
    # Save pose if best stored result is lower in energy
    if E < Eb:
        best.assign(tripeptide)
        Eb = E

We plot how the system energy changes along the minimization algorithm

In [ ]:
plt.plot(energies)
plt.title('Simple minimization')
plt.xlabel('Step')
plt.ylabel('Energy [kcal/mol]')

We see that in under 500 steps, our method has already converged to the nearest local minima. We can check the progression of the trajectory by plotting the phi and psi torsional values:

In [ ]:
# Define resolution and figure size
plt.figure(dpi=70, figsize=(4,4))

# Plot the phi and psi values as a scatter plot
plt.scatter(phi_values, psi_values, c='k', s=0.1)

# Generate labels for each axis
plt.xlabel('$\phi$')
plt.ylabel('$\psi$')

# Define the plot x and y limits
plt.xlim(-180,180)
plt.ylim(-180,180)

# Set a title
plt.title('Simple minimization')

In [ ]:
print('The best sampled score was: %.2f kcal/mol' % Eb)
pymol_mover.apply(best)

The exploration is stuck in the local minima, and it won't be able to escape this point since every perturbation will increase the conformation's energy.

### Monte-Carlo sampling

We now set up a Monte-Carlo sampling strategy for our tripeptide system. First, let's define the Monte-Carlo function:

In [ ]:
def monteCarlo(pose, mover, score_function, temperature=0.5):
    
    # Get the current energy of the pose
    E0 = score_function(pose)
    
    # Create a copy of the pose
    clone_pose = Pose()
    clone_pose.assign(pose)
    
    # Apply perturbation to cloned pose
    mover(clone_pose)
    
    # Evaluate energy of the perturbed pose
    E1 = score_function(clone_pose)
    
    # Calculate the acceptance probability
    P = np.min([1, np.exp(-(E1-E0)/temperature)])
    
    if P >= np.random.uniform(low=0, high=1.0):
        pose.assign(clone_pose)
        
        return 1
        
    return 0

Now, we start the sampling for an equal 100000 steps:

In [ ]:
# Redefine the tripeptide Pose
tripeptide = pose_from_sequence("AAA")

# Set phi and psi values to 90
tripeptide.set_phi(2, 90)
tripeptide.set_psi(2, 90)

# Store energies into a list
energies = []

# Store phi and psi torsion angles into a list
phi_values = []
psi_values = []

n_steps = 100000
accepted = 0

# Create pose to store best sampled result
best = Pose()
best.assign(tripeptide)
Eb = sfxn(best)

for i in range(n_steps): 
    accepted += monteCarlo(tripeptide, perturb_random_angle, sfxn)
    
    E = sfxn(tripeptide)
    energies.append(E)
    
    phi_values.append(tripeptide.phi(2))
    psi_values.append(tripeptide.psi(2))
    
    # Save pose if best stored result is lower in energy
    if E < Eb:
        best.assign(tripeptide)
        Eb = E
    
print('Accepted fraction %s' % (accepted/n_steps))

Let's now plot the energy progression of our short MC sampling:

In [ ]:
plt.plot(energies)
plt.title('Monte-Carlo minimization (T=0.5)')
plt.xlabel('Step')
plt.ylabel('Energy [kcal/mol]')

The system tends to explore low-energy regions of the landscape; however, the method can explore several minima since it sometimes accepts moves in the opposite direction (higher energy conformations). 

In [ ]:
# Define resolution and figure size
plt.figure(dpi=70, figsize=(4,4))

# Plot the phi and psi values as a scatter plot
plt.scatter(phi_values, psi_values, c='k', s=0.1)

# Generate labels for each axis
plt.xlabel('$\phi$')
plt.ylabel('$\psi$')

# Define the plot x and y limits
plt.xlim(-180,180)
plt.ylim(-180,180)

# Set a title
plt.title('Monte-Carlo sampling (T=0.5)')

In [ ]:
print('The best sampled score was: %.2f kcal/mol' % Eb)
pymol_mover.apply(best)

Let's repeat our MC sampling but now increasing the temperature criteria to calculate the acceptance probability:

In [ ]:
# Redefine the tripeptide Pose
tripeptide = pose_from_sequence("AAA")

# Set phi and psi values to 90
tripeptide.set_phi(2, 90)
tripeptide.set_psi(2, 90)

# Store energies into a list
energies = []

# Store phi and psi torsion angles into a list
phi_values = []
psi_values = []

# Create pose to store best sampled result
best = Pose()
best.assign(tripeptide)
Eb = sfxn(best)

n_steps = 100000
accepted = 0

for i in range(n_steps): 
    accepted += monteCarlo(tripeptide, perturb_random_angle, sfxn, temperature=3)
    
    E = sfxn(tripeptide)
    energies.append(E)
    
    phi_values.append(tripeptide.phi(2))
    psi_values.append(tripeptide.psi(2))
    
    # Save pose if best stored result is lower in energy
    if E < Eb:
        best.assign(tripeptide)
        Eb = E
    
print('Accepted fraction %s' % (accepted/n_steps))

In [ ]:
plt.plot(energies)
plt.title('Monte-Carlo minimization (T=3)')
plt.xlabel('Step')
plt.ylabel('Energy [kcal/mol]')

In [ ]:
# Define resolution and figure size
plt.figure(dpi=70, figsize=(4,4))

# Plot the phi and psi values as a scatter plot
plt.scatter(phi_values, psi_values, c='k', s=0.1)

# Generate labels for each axis
plt.xlabel('$\phi$')
plt.ylabel('$\psi$')

# Define the plot x and y limits
plt.xlim(-180,180)
plt.ylim(-180,180)

# Set a title
plt.title('Monte-Carlo sampling (T=3)')

In [ ]:
print('The best sampled score was: %.2f kcal/mol' % Eb)
pymol_mover.apply(best)

We see that increasing the temperature now affects the magnitude of the energy-increasing steps. With bigger changes in energy in the opposite direction, the method can jump more considerable energy barriers and explore a larger surface of the energy landscape. This broader sampling will be done at the expense of exploring more poorly the regions of lower energy. It is, therefore, crucial to find a temperature that maintains a good trade-off between sampling low energy conformations and still being able to jump energy barriers between energy minima.

### Simulated annealing MC sampling

One strategy to solvent the temperature effect is to start with a high temperature and then reduce it as the simulation progresses. This strategy is called simulated annealing. Let's run a short simulated annealing strategy to search the torsional space of our tripeptide:

In [ ]:
# Simulated annealing parameters

Ti = 3
Tf = 0.5

# Redefine the tripeptide Pose
tripeptide = pose_from_sequence("AAA")

# Set phi and psi values to 90
tripeptide.set_phi(2, 90)
tripeptide.set_psi(2, 90)

# Store energies into a list
energies = []

# Store phi and psi torsion angles into a list
phi_values = []
psi_values = []

# Create pose to store best sampled result
best = Pose()
best.assign(tripeptide)
Eb = sfxn(best)

n_steps = 100000
accepted = 0

for i in range(n_steps): 
    
    T = Ti + i*((Tf - Ti)/(n_steps-1)) # Decrease the acceptance temperature
    
    accepted += monteCarlo(tripeptide, perturb_random_angle, sfxn, temperature=T)
    
    E = sfxn(tripeptide)
    energies.append(E)
    
    phi_values.append(tripeptide.phi(2))
    psi_values.append(tripeptide.psi(2))
    
    # Save pose if best stored result is lower in energy
    if E < Eb:
        best.assign(tripeptide)
        Eb = E
    
print('Accepted fraction %s' % (accepted/n_steps))

In [ ]:
plt.plot(energies)
plt.title('Simulated annealing Monte-Carlo minimization ($T_i$=3 and $T_f$=0.5)')
plt.xlabel('Step')
plt.ylabel('Energy [kcal/mol]')

In [ ]:
# Define resolution and figure size
plt.figure(dpi=70, figsize=(4,4))

# Plot the phi and psi values as a scatter plot
plt.scatter(phi_values, psi_values, c='k', s=0.1)

# Generate labels for each axis
plt.xlabel('$\phi$')
plt.ylabel('$\psi$')

# Define the plot x and y limits
plt.xlim(-180,180)
plt.ylim(-180,180)

# Set a title
plt.title('Simulated annealing Monte-Carlo minimization ($T_i$=3 and $T_f$=0.5)')

In [ ]:
print('The best sampled score was: %.2f kcal/mol' % Eb)
pymol_mover.apply(best)

### Cyclic simulated annealing MC sampling

Another strategy is to change the temperature of the simulation in a cyclic fashion. We add a new variable which is the length of one temperature shift cycle:

In [ ]:
# Simulated annealing parameters

T_high = 3
T_low = 0.5
period = 5000

# Redefine the tripeptide Pose
tripeptide = pose_from_sequence("AAA")

# Set phi and psi values to 90
tripeptide.set_phi(2, 90)
tripeptide.set_psi(2, 90)

# Store energies into a list
energies = []
temperatures = []

# Store phi and psi torsion angles into a list
phi_values = []
psi_values = []

# Create pose to store best sampled result
best = Pose()
best.assign(tripeptide)
Eb = sfxn(best)

n_steps = 100000
accepted = 0

for i in range(n_steps): 
    
    sin_fraction =  np.sin(i/(period/4)/2*np.pi) # Convert steps into a sin function with the correct period
    T = T_low + (T_high-T_low)*((sin_fraction)+1)/2 # Multiply for dT to adjust T accordingly
    
    temperatures.append(T)
    
    accepted += monteCarlo(tripeptide, perturb_random_angle, sfxn, temperature=T)
    
    E = sfxn(tripeptide)
    energies.append(E)
    
    phi_values.append(tripeptide.phi(2))
    psi_values.append(tripeptide.psi(2))
    
    # Save pose if best stored result is lower in energy
    if E < Eb:
        best.assign(tripeptide)
        Eb = E
    
print('Accepted fraction %s' % (accepted/n_steps))

In [ ]:
plt.plot(temperatures)
plt.title('Cyclic simulated annealing Monte-Carlo sampling ($T_{high}$=3 and $T_{low}$=0.5)')
plt.xlabel('Step')
plt.ylabel('Temperature')

In [ ]:
plt.plot(energies)
plt.title('Cyclic simulated annealing Monte-Carlo sampling ($T_{high}$=3 and $T_{low}$=0.5)')
plt.xlabel('Step')
plt.ylabel('Energy [kcal/mol]')

In [ ]:
# Define resolution and figure size
plt.figure(dpi=70, figsize=(4,4))

# Plot the phi and psi values as a scatter plot
plt.scatter(phi_values, psi_values, c='k', s=0.1)

# Generate labels for each axis
plt.xlabel('$\phi$')
plt.ylabel('$\psi$')

# Define the plot x and y limits
plt.xlim(-180,180)
plt.ylim(-180,180)

# Set a title
plt.title('My first Ramachandran plot')

In [ ]:
print('The best sampled score was: %.2f kcal/mol' % Eb)
pymol_mover.apply(best)

The temperature of this sampling scheme increases and decreases in a cyclic fashion. Assume the time to complete a cycle and the temperature range are well-calibrated. In that case, the method can profit from jumping high energy barriers when exploring at high temperatures and sample low-energy conformations when at low temperatures.